# Grid2x2 using RLlib

In [1]:
import csv
import os

import ray
from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility
from ray.tune.registry import register_env

from envs import MultiAgentSumoEnv
from observation import Grid2x2ObservationFunction
from reward_functions import combined_reward

In [2]:
import random
import numpy as np
import torch

TEST_NUM = 4
SEED = 23423  # default SUMO seed no.
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

ENV_NAME = "grid2x2"

In [3]:
def train_env_creator(args):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": Grid2x2ObservationFunction,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(**env_params)
    return env

In [4]:
from ray.rllib.algorithms.ppo import PPOConfig

# From https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/ppo/atari-ppo.yaml

train_env = MultiAgentEnvCompatibility(train_env_creator({}))

config: PPOConfig
config = (
    PPOConfig()
    .environment(env=ENV_NAME)
    .framework(framework="torch")
    .rollouts(
        rollout_fragment_length=100,
        num_rollout_workers=10,
    )
    .training(
        lambda_=0.95,
        kl_coeff=0.5,
        clip_param=0.1,
        vf_clip_param=10.0,
        entropy_coeff=0.01,
        train_batch_size=1000,
        sgd_minibatch_size=100,
        num_sgd_iter=10,
    )
    .evaluation(
        evaluation_duration=1,
        evaluation_num_workers=1,
        evaluation_sample_timeout_s=300,
    )
    .debugging(seed=SEED)
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "1")))
    .multi_agent(
        policies=set(train_env.env.ts_ids),
        policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
    )
    .fault_tolerance(recreate_failed_workers=True)
)

delta_wait_time_reward 0.8


In [5]:
config.recreate_failed_workers

True

In [6]:
csv_dir = os.path.join("outputs",ENV_NAME,f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

## Play Untrained Agent

In [7]:
def eval_env_creator(csv_path, tb_log_dir):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": Grid2x2ObservationFunction,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(eval=True, csv_path=csv_path, tb_log_dir=tb_log_dir, **env_params)
    return env

In [8]:
ray.init()

csv_path = os.path.join(csv_dir, "untrained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_untrained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure",])

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

2023-06-04 00:24:08,914	INFO worker.py:1625 -- Started a local Ray instance.


In [9]:
algo = config.build()

2023-06-04 00:24:12,746	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-04 00:24:12,840	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=18072) delta_wait_time_reward 0.8
(RolloutWorker pid=18072) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                      


(RolloutWorker pid=7352) 2023-06-04 00:24:20,131	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-04 00:24:24,017	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=25996) delta_wait_time_reward 0.8 [repeated 10x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RolloutWorker pid=11628) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 9x across cluster]


(RolloutWorker pid=25996) 2023-06-04 00:24:28,256	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-04 00:24:28,349	INFO trainable.py:172 -- Trainable.setup took 15.526 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [10]:
algo.evaluate()

Step #3600.00 (1ms ~= 1000.00*RT, ~223000.00UPS, TraCI: 248ms, vehicles TOT 1282 ACT 223 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~223000.00UPS, TraCI: 248ms, vehicles TOT 1282 ACT 223 BOT 3 ACT 3 BUF 0)                      


{'evaluation': {'episode_reward_max': -457220.68761799985,
  'episode_reward_min': -457220.68761799985,
  'episode_reward_mean': -457220.68761799985,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'1': -114175.06977694454,
   '2': -123713.48071310198,
   '5': -107213.35747978264,
   '6': -112118.77964817011},
  'policy_reward_max': {'1': -114175.06977694454,
   '2': -123713.48071310198,
   '5': -107213.35747978264,
   '6': -112118.77964817011},
  'policy_reward_mean': {'1': -114175.06977694454,
   '2': -123713.48071310198,
   '5': -107213.35747978264,
   '6': -112118.77964817011},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [-457220.68761799985],
   'episode_lengths': [720],
   'policy_1_reward': [-114175.06977694454],
   'policy_2_reward': [-123713.48071310198],
   'policy_5_reward': [-107213.35747978264],
   'policy_6_reward': [-112118.77964817011]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 7.0663103614203

In [11]:
ray.shutdown()

## Train RL Agent

In [12]:
ray.init()

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(train_env_creator(config)))

2023-06-04 00:29:16,896	INFO worker.py:1625 -- Started a local Ray instance.


In [13]:
algo = config.build()

2023-06-04 00:29:20,225	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=30388) delta_wait_time_reward 0.8


(RolloutWorker pid=30388) 2023-06-04 00:29:26,745	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.


(RolloutWorker pid=30388) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 0 ACT 0 BUF 0)                      


2023-06-04 00:29:27,053	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-04 00:29:31,074	INFO trainable.py:172 -- Trainable.setup took 10.844 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [14]:
from datetime import datetime

TRAIN_EPS = 1400  # 720 * 1400 == 1_008_000 total timesteps
CHECKPOINT_FREQ = 100
assert TRAIN_EPS % CHECKPOINT_FREQ == 0

tic = datetime.now()

for i in range(TRAIN_EPS):
    results = algo.train()

    if (i+1) % CHECKPOINT_FREQ == 0:
        algo.save(os.path.join("ray_checkpoints","grid2x2",f"test_{TEST_NUM}"))

toc = datetime.now()

2023-06-04 00:29:44,852	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


Step #3600.00 (1ms ~= 1000.00*RT, ~99000.00UPS, TraCI: 99ms, vehicles TOT 2106 ACT 99 BUF 24ms, vehicles TOT 3 ACT 3 BUF 0)      
(RolloutWorker pid=32616) delta_wait_time_reward 0.8 [repeated 10x across cluster]
(RolloutWorker pid=32616) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 10x across cluster]
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 93ms, vehicles TOT 2104 ACT 103 BUF 2)              26ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~123000.00UPS, TraCI: 93ms, vehicles TOT 2104 ACT 123 BU27ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 100ms, vehicles TOT 2095 ACT 140 BUF 11)            OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~173000.00UPS, TraCI: 100ms, vehicles TOT 2087 ACT 173 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~184000.00UPS, TraCI: 100ms, vehicles TOT 2099 ACT 184 B24ms, vehicles TOT 3 ACT 3 BUF 

(RolloutWorker pid=32584) 2023-06-04 01:12:42,354	ERROR actor_manager.py:179 -- Worker exception, recreating: unpack requires a buffer of 3 bytes
(RolloutWorker pid=32584) Traceback (most recent call last):
(RolloutWorker pid=32584)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\utils\actor_manager.py", line 176, in apply
(RolloutWorker pid=32584)     return func(self, *args, **kwargs)
(RolloutWorker pid=32584)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\execution\rollout_ops.py", line 86, in <lambda>
(RolloutWorker pid=32584)     lambda w: w.sample(), local_worker=False, healthy_only=True
(RolloutWorker pid=32584)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
(RolloutWorker pid=32584)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=32584)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 

(RolloutWorker pid=3984) delta_wait_time_reward 0.8
(RolloutWorker pid=3984) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~46000.00UPS, TraCI: 90ms, vehicles TOT 2106 ACT 46 BUF 24ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 86ms, vehicles TOT 2106 ACT 51 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 83ms, vehicles TOT 2106 ACT 52 BUF 0)               19ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~46000.00UPS, TraCI: 78ms, vehicles TOT 2106 ACT 46 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 82ms, vehicles TOT 2106 ACT 52 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~55000.00UPS, TraCI: 82ms, vehicles TOT 2106 ACT 55 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 83ms, vehicles 

(RolloutWorker pid=17344) Error: tcpip::Socket::accept() Unable to create listening socket: Address already in use
(RolloutWorker pid=17344) Quitting (on error).
(RolloutWorker pid=17344) C:\Program Files (x86)\Eclipse\Sumo\tools\traci\main.py:159: UserWarning: Could not connect to TraCI server using port 60756 (TraCI server already finished). Retrying with different port.
(RolloutWorker pid=17344)   warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


Step #3600.00 (1ms ~= 1000.00*RT, ~43000.00UPS, TraCI: 73ms, vehicles TOT 2106 ACT 43 BUF 7ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~48000.00UPS, TraCI: 97ms, vehicles TOT 2106 ACT 48 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 95ms, vehicles TOT 2106 ACT 42 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~41000.00UPS, TraCI: 91ms, vehicles TOT 2106 ACT 41 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 101ms, vehicles TOT 2106 ACT 44 BUF 0)              19ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 94ms, vehicles TOT 2106 ACT 49 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~50000.00UPS, TraCI: 65ms, vehicles TOT 2106 ACT 50 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 71ms, vehicles TOT 2106 ACT 48 BUF 0)               O

(RolloutWorker pid=20208) Quitting (on error).
(RolloutWorker pid=20208) Quitting (on error).
(RolloutWorker pid=15472) Error: tcpip::Socket::accept() Unable to create listening socket: Address already in use
(RolloutWorker pid=15472) Quitting (on error).
(RolloutWorker pid=20208) C:\Program Files (x86)\Eclipse\Sumo\tools\traci\main.py:159: UserWarning: Could not connect to TraCI server using port 61948 (TraCI server already finished). Retrying with different port.
(RolloutWorker pid=20208)   warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


Step #3600.00 (1ms ~= 1000.00*RT, ~49000.00UPS, TraCI: 90ms, vehicles TOT 2106 ACT 49 BUF T 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~40000.00UPS, TraCI: 76ms, vehicles TOT 2106 ACT 40 BUF 18ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~49000.00UPS, TraCI: 79ms, vehicles TOT 2106 ACT 49 BUF 18ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~44000.00UPS, TraCI: 79ms, vehicles TOT 2106 ACT 44 BUF 20ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~46000.00UPS, TraCI: 82ms, vehicles TOT 2106 ACT 46 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~47000.00UPS, TraCI: 81ms, vehicles TOT 2106 ACT 47 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 59ms, vehicles TOT 2106 ACT 44 BUF 0)               21ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 66ms, vehicles TOT 2106 ACT 44 BUF 0)               1

(RolloutWorker pid=28548) Quitting (on error).
(RolloutWorker pid=28548) Quitting (on error).
(RolloutWorker pid=15472) Quitting (on error).
(RolloutWorker pid=15472) Quitting (on error).
(RolloutWorker pid=28548) C:\Program Files (x86)\Eclipse\Sumo\tools\traci\main.py:159: UserWarning: Could not connect to TraCI server using port 62962 (TraCI server already finished). Retrying with different port.
(RolloutWorker pid=28548)   warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


Step #3600.00 (1ms ~= 1000.00*RT, ~47000.00UPS, TraCI: 67ms, vehicles TOT 2106 ACT 47 BUF T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 87ms, vehicles TOT 2106 ACT 44 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 85ms, vehicles TOT 2106 ACT 44 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~42000.00UPS, TraCI: 75ms, vehicles TOT 2106 ACT 42 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 78ms, vehicles TOT 2106 ACT 42 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 67ms, vehicles TOT 2106 ACT 49 BUF 0)               20ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~45000.00UPS, TraCI: 75ms, vehicles TOT 2106 ACT 45 BUF 19ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 81ms, vehicles TOT 2106 ACT 45 BUF 0)               O

In [15]:
toc - tic

datetime.timedelta(seconds=20125, microseconds=283738)

In [16]:
ray.shutdown()

## Play Trained Agent

In [17]:
csv_path = os.path.join(csv_dir, "trained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_trained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure",])

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

In [18]:
ray.init()

2023-06-04 06:05:02,131	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.8.16
Ray version:,2.4.0


In [19]:
from ray.rllib.algorithms.ppo import PPO

checkpoint_path = os.path.join("ray_checkpoints",ENV_NAME,f"test_{TEST_NUM}",f"checkpoint_{TRAIN_EPS:06}")
checkpoint_path = os.path.abspath(checkpoint_path)
print(checkpoint_path)

ppo_agent = PPO.from_checkpoint(checkpoint_path)

2023-06-04 06:05:05,705	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory c:\Users\admin\masters-proj\ray_checkpoints\grid2x2\test_4\checkpoint_001400! Trying to extract checkpoint info from other files found in that dir.
2023-06-04 06:05:05,742	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


c:\Users\admin\masters-proj\ray_checkpoints\grid2x2\test_4\checkpoint_001400
(RolloutWorker pid=22992) delta_wait_time_reward 0.8


(RolloutWorker pid=27856) 2023-06-04 06:05:12,684	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-04 06:05:12,926	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=18768) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 0 ACT 0 BUF 0)                      


2023-06-04 06:05:16,940	INFO trainable.py:172 -- Trainable.setup took 11.193 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [20]:
ppo_agent.evaluate()

Step #3600.00 (1ms ~= 1000.00*RT, ~44000.00UPS, TraCI: 160ms, vehicles TOT 2106 ACT 44 BUF19ms, vehicles TOT 3 ACT 3 BUF 0)      
(RolloutWorker pid=27184) delta_wait_time_reward 0.8 [repeated 10x across cluster]
(RolloutWorker pid=27184) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 10x across cluster]


{'evaluation': {'episode_reward_max': -81509.2120899747,
  'episode_reward_min': -81509.2120899747,
  'episode_reward_mean': -81509.2120899747,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'1': -19665.3491720903,
   '2': -20932.82990029217,
   '5': -19711.23768465219,
   '6': -21199.795332939837},
  'policy_reward_max': {'1': -19665.3491720903,
   '2': -20932.82990029217,
   '5': -19711.23768465219,
   '6': -21199.795332939837},
  'policy_reward_mean': {'1': -19665.3491720903,
   '2': -20932.82990029217,
   '5': -19711.23768465219,
   '6': -21199.795332939837},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [-81509.2120899747],
   'episode_lengths': [720],
   'policy_1_reward': [-19665.3491720903],
   'policy_2_reward': [-20932.82990029217],
   'policy_5_reward': [-19711.23768465219],
   'policy_6_reward': [-21199.795332939837]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 6.999624254964758,
   'mean_inference_m

In [21]:
ray.shutdown()

(RolloutWorker pid=27184) 2023-06-04 06:05:16,837	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
